In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/object_detection/utils')

file_list = os.listdir()
print(file_list)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['coco_utils.py', 'engine.py', 'transforms.py', 'utils.rar', 'utils.py', 'coco_eval.py', '__pycache__', 'fix1_model', 'vision']


In [2]:
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

from PIL.ExifTags import TAGS
from engine import train_one_epoch,evaluate
import utils
from torchvision import transforms as T

In [3]:
class DataSet(torch.utils.data.Dataset):

    def __init__(self,root, data_file, transform):
        self.root = root ### mapa cu fotografii
        self.transform = transform
        self.data = pd.read_csv(data_file, sep=';')
        self.imgs = self.data['image_path'].values
        self.path_to_data_file = data_file

    def __getitem__(self,idx):

        img_path = os.path.join(self.root,self.imgs[idx])
        img = Image.open(img_path).convert('RGB')


        for orientation in TAGS.keys() :
            if TAGS[orientation]=='Orientation' : break
        # Getting the exif
        exif=dict(img.getexif().items())
        # Rotating the image if the orientation is wrong.
        if len(exif)!=0:
            if orientation in exif.keys():
                if exif[orientation] == 3 :
                    img=img.rotate(180, expand=True)
                elif exif[orientation] == 6 :
                    img=img.rotate(270, expand=True)
                elif exif[orientation] == 8 :
                    img=img.rotate(90, expand=True)

        box_list = self.data[self.data['image_path']==self.imgs[idx]][['x_min','y_min','x_max','y_max']].values
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_obj = len(box_list)
        labels = torch.ones((num_obj,), dtype = torch.int64)
        iscrowd = torch.zeros((num_obj,), dtype=torch.int64)
        area = (boxes[:,3] - boxes[:, 1]) * (boxes[:,2] - boxes[:, 0])

        image_id = torch.tensor([idx])

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['area'] = area
        target['iscrowd'] = iscrowd
        target['image_id'] = image_id

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.data)

In [4]:
transformations = T.Compose([T.ToTensor()])
dataset = DataSet('/content/drive/MyDrive/object_detection/data',
                  '/content/drive/MyDrive/object_detection/glasses2.csv', transformations)
torch.manual_seed(1)

indices = torch.randperm(len(dataset)).tolist()
train = torch.utils.data.Subset(dataset, indices[:100])
test = torch.utils.data.Subset(dataset, indices[100:])

train_loader = torch.utils.data.DataLoader(train, batch_size = 1, shuffle=True, collate_fn = utils.collate_fn)
test_loader = torch.utils.data.DataLoader(test, batch_size = 1, shuffle=False, collate_fn = utils.collate_fn)



In [5]:
dataset[1]

(tensor([[[0.7294, 0.7294, 0.7294,  ..., 0.5412, 0.5412, 0.5412],
          [0.7294, 0.7294, 0.7294,  ..., 0.5412, 0.5412, 0.5412],
          [0.7294, 0.7294, 0.7294,  ..., 0.5412, 0.5412, 0.5412],
          ...,
          [0.5922, 0.5922, 0.5922,  ..., 0.4000, 0.4000, 0.4000],
          [0.5922, 0.5922, 0.5922,  ..., 0.4000, 0.4000, 0.4000],
          [0.5922, 0.5922, 0.5922,  ..., 0.4000, 0.4000, 0.4000]],
 
         [[0.7412, 0.7412, 0.7412,  ..., 0.6078, 0.6078, 0.6078],
          [0.7412, 0.7412, 0.7412,  ..., 0.6078, 0.6078, 0.6078],
          [0.7412, 0.7412, 0.7412,  ..., 0.6078, 0.6078, 0.6078],
          ...,
          [0.6549, 0.6549, 0.6549,  ..., 0.4863, 0.4863, 0.4863],
          [0.6549, 0.6549, 0.6549,  ..., 0.4863, 0.4863, 0.4863],
          [0.6549, 0.6549, 0.6549,  ..., 0.4863, 0.4863, 0.4863]],
 
         [[0.8000, 0.8000, 0.8000,  ..., 0.6706, 0.6706, 0.6706],
          [0.8000, 0.8000, 0.8000,  ..., 0.6706, 0.6706, 0.6706],
          [0.8000, 0.8000, 0.8000,  ...,

In [6]:
def get_model(num_clase):

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    for param in model.parameters():
        param.required_grad = False

    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_clase)

    return model

In [7]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
num_clase = 2
model = get_model(num_clase).to(DEVICE)

params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.SGD(params, lr = 0.005, momentum=0.9, weight_decay = 0.0005)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
num_epoch = 5

for epoch in range(num_epoch):

    train_one_epoch(model, optimizer, train_loader, DEVICE, epoch, print_freq = 10)

    evaluate(model, test_loader, device=DEVICE)

Epoch: [0]  [  0/100]  eta: 0:05:56  lr: 0.000055  loss: 0.7732 (0.7732)  loss_classifier: 0.7057 (0.7057)  loss_box_reg: 0.0132 (0.0132)  loss_objectness: 0.0433 (0.0433)  loss_rpn_box_reg: 0.0110 (0.0110)  time: 3.5611  data: 0.1331  max mem: 962
Epoch: [0]  [ 10/100]  eta: 0:00:54  lr: 0.000560  loss: 0.5489 (0.5223)  loss_classifier: 0.4341 (0.4140)  loss_box_reg: 0.0313 (0.0557)  loss_objectness: 0.0433 (0.0446)  loss_rpn_box_reg: 0.0077 (0.0080)  time: 0.6099  data: 0.0468  max mem: 1520
Epoch: [0]  [ 20/100]  eta: 0:00:37  lr: 0.001065  loss: 0.2151 (0.3573)  loss_classifier: 0.0997 (0.2592)  loss_box_reg: 0.0436 (0.0542)  loss_objectness: 0.0155 (0.0356)  loss_rpn_box_reg: 0.0052 (0.0083)  time: 0.3120  data: 0.0507  max mem: 1568
Epoch: [0]  [ 30/100]  eta: 0:00:28  lr: 0.001569  loss: 0.1840 (0.3082)  loss_classifier: 0.0926 (0.2109)  loss_box_reg: 0.0624 (0.0633)  loss_objectness: 0.0123 (0.0270)  loss_rpn_box_reg: 0.0038 (0.0070)  time: 0.2968  data: 0.0580  max mem: 1609
E

AttributeError: ignored

In [ ]:
torch.save(model.state_dict(),r'D:\learning\pytorch\nets_learning\object_detection\fix1_ports_model.pth')



In [ ]:
# Use this code to made a predition.
from PIL import ImageDraw
# Getting the image.
for idx in range(2,20):
    img, _ = test[idx]
    # Getting the object coordinates.
    label_boxes = np.array(test[idx][1]['boxes'])

    # Setting the model to eval state.
    loaded_model.eval()
    # Making the prediction.
    with torch.no_grad():
        prediction = loaded_model([img])

    # Getting an drawing the image.
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)

    # Drawing the real box around the object.
    for elem in range(len(label_boxes)):
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
                       (label_boxes[elem][2], label_boxes[elem][3])],
                      outline='green', width=3)
    # Drawing the predicted box around the object.
    for element in range(len(prediction[0]['boxes'])):
        boxes = prediction[0]['boxes'][element].cpu().numpy()
        score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals=4)

        if score > 0.2:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])],
                         outline='red', width=3)
            draw.text((boxes[0], boxes[1]), text=str(score))
    display(image)